In [1]:
import json
from BuildGraphModule import *
from unidecode import unidecode
from graviz import graph_processing

In [2]:
with open('paper.json', 'r') as f:
    data = json.load(f)

In [3]:
# Uniform strings - Authors
for i in list(data.keys()):
    try:
        for j in range(len(data[i]['Authors'])):
            t = unidecode(data[i]['Authors'][j])
            t.encode("ascii")
            data[i]['Authors'][j] = t.lower().replace('.','')
            
    except:
        continue

In [4]:
# Uniform strings - EPFL Authors
for i in list(data.keys()):
    dic = []
    try:
        for j in range(len(data[i]['Epfl authors'])):
            string = list(data[i]['Epfl authors'][j].keys())[0]
            t = unidecode(string)
            t.encode("ascii")
            dic += [{t.lower().replace('.','') : list(data[i]['Epfl authors'][j].values())[0]}]
        data[i]['Epfl authors'] = dic
    except:
        continue

In [ ]:
authors = get_authors(data, 'Authors')

In [ ]:
# All
dictionary_a_id, dictionary_id_a = create_authors_id(authors, 'all')

In [ ]:
# All
dict_authors = list_coauthors(data)

In [8]:
import time
s = time.time()
# All
dict_numb_coll = number_collaborations(dict_authors)
print (time.time()-s)

In [9]:
# All
dict_authors_set = set_coauthors(dict_authors)

In [10]:
# All
edges = create_edges(dict_authors_set, dictionary_a_id)

## EPFL graph: Multiplegraph

In [11]:
authors_epfl = get_authors(data, 'Epfl authors')
# Due to the format which the authors have been saved

# We initialize a new list
epfl_authors = []

# Then for each element in the previous list
for i in authors_epfl:
    # We unpack the content
    epfl_authors += list(i.keys())

In [12]:
# Id-Author / Author-ID
dictionary_a_id_epfl, dictionary_id_a_epfl = create_authors_id(epfl_authors, 'epfl')

In [13]:
# EPFL - COAUTORI NELLA LISTA DI EPFL AUTHORS
dict_authors_epfl = list_coauthors_epfl(data)

In [14]:
# Number of collaborations
dict_numb_coll_epfl = number_collaborations(dict_authors_epfl)

In [15]:
# Coauthors set
dict_authors_set_epfl = set_coauthors(dict_authors_epfl)

In [16]:
# Set of edges
edges_epfl = create_edges(dict_authors_set_epfl, dictionary_a_id_epfl)

In [46]:
G = create_simple_graph(dictionary_id_a_epfl, edges_epfl)
nx.set_node_attributes(G, 'author_id', dictionary_id_a_epfl)
graph_processing.save_graph(G, 'graph.json', 'JSON')

Name: 
Type: Graph
Number of nodes: 13451
Number of edges: 45417
Average degree:   6.7530


In [ ]:
# Initialize the dictionary
dict_authors_epfl = defaultdict(list)

# For each pubblication
for p in list(data.keys()):
    # If present
    try:
        # Extracte the list of epfl authors
        list_authors_raw = data[p]['Epfl authors']
        # Unpack the list
        list_authors_list = [list(i.keys()) for i in list_authors_raw]

        # Whether the list of authors contains more than one element
        if len(list_authors_list) > 1:
            # Get the final list unpacking elements
            list_authors = [i[0] for i in list_authors_list]
        else:
            # Just get the element as the list
            list_authors = list_authors_list[0]

        # For each author create an instance in the dictionary
        for a in list_authors:
            dict_authors_epfl[a] += [{j:p} for j in list_authors if j != a]

    # If there re not epfl authors continue the loop
    except:
        continue

In [ ]:
dict_authors_epfl

defaultdict(list,
            {'membrez yves': [{'holliger christof': '182490'},
              {'fruteau de laclos helene': '182490'},
              {'deront marc': '182490'}],
             'dunant cyrille': [{'cuba ramos aurelia isabel': '217336'},
              {'molinari jean-francois': '217336'},
              {'cuba ramos aurelia isabel': '217337'},
              {'molinari jean-francois': '217337'},
              {'scrivener karen': '140994'},
              {'giorla alain benjamin': '189927'},
              {'scrivener karen': '189927'}],
             'dallavilla tiziano': [{'savoglidis georgios': '217549'},
              {'hatzimanikatis vassily': '217549'}],
             'hediger s': [{'sayah a': '55966'},
              {'van der goot fg': '55966'},
              {'gijs mam': '55966'},
              {'sayah a': '55993'},
              {'gijs m a m': '55993'},
              {'sayah a': '55992'},
              {'gijs m a m': '55992'},
              {'sayah a': '55970'},
         

In [14]:
dict_coautorship_papers = {}
for author in list(dict_authors_epfl.keys()):
    
    dict_papers = defaultdict(list)
    for i in range(len(dict_authors_epfl[author])):

        coauthor = list(dict_authors_epfl[author][i].keys())[0]

        dict_papers[coauthor] += [(dict_authors_epfl[author][i])[coauthor]]

    dict_coautorship_papers[author] = dict_papers

In [15]:
dictionary = {}
dictionary_occ = {}
for i in list(dict_coautorship_papers.keys()):
    dictionary[i] = {}
    dictionary_occ[i] = {}
    for j in list(dict_coautorship_papers[i].keys()):
        dates = defaultdict(list)
        for l in range(len(dict_coautorship_papers[i][j])):
            try:
                dates[data[sorted(dict_coautorship_papers[i][j])[l]]['Publication date']] += [sorted(dict_coautorship_papers[i][j])[l]]
            except:
                continue
        if len(dates) != 0:
            dictionary[i][j] = dates
            dictionary_occurences = {}
            for k,m in dictionary[i][j].items():
                dictionary_occurences[k] = len(m)
            dictionary_occ[i][j] = dictionary_occurences

In [16]:
# Get list of paper for author
list_paper = defaultdict(list)
for author in list(dict_coautorship_papers.keys()):
    list_paper[author] += [dict_coautorship_papers[author][i] for i in dict_coautorship_papers[author]]
for a in list(list_paper.keys()):
    list_paper[a] = set([j for i in list_paper[a] for j in i])

In [17]:
import requests
from bs4 import BeautifulSoup

In [ ]:
author_lab = {}
site_lab = {}
for a in list(list_paper.keys()):
    try:
        for p in list_paper[a]:
            if len(data[p]['Labs involved']) <= 1:
                author_lab[a] =  list(data[p]['Labs involved'][0].keys())[0]
                
                break
            else:
                print ('else')
                req = requests.get(dict_author_site[a])
                html = req.content
                soup = BeautifulSoup(html, 'html.parser')
                #print (soup)
                try:
                    #print ('try')
                    for i in soup.findAll('div', {'class':'topaccred'})[0]:
                        
                        try:
                            author_lab[a] = site_lab[i]
                            break
                        except:
                            print (i.get('href'))
                            req = requests.get(i.get('href'))
                            html = req.content
                            soup = BeautifulSoup(html, 'html.parser')
                            author_lab[a] = soup.findAll('h2')[0].text.replace('\n',' ').split('  ')[0]
                            site_lab[i] = soup.findAll('h2')[0].text.replace('\n',' ').split('  ')[0]
                            break
                    
                
                except:
                    continue
            
    except:
        continue

else
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
	"https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xml:lang="fr" xmlns="https://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<title>
  	Tiziano Dallavilla : Contacts
  </title>
<!-- web2010 EPFL -->
<link href="https://www.epfl.ch/css/epfl.css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://www.epfl.ch/css/applications.css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://www.epfl.ch/css/print.css" media="print" rel="stylesheet" type="text/css"/>
<link href="https://www.epfl.ch/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<script src="https://www.epfl.ch/js/jquery-epfl.min.js" type="text/javascript"></script>
<script type="text/javascript">jQuery.noConflict();</script>
<script src="https://www.epfl.ch/js/globalnav.js" type="text/javascript"></script>
<!-- people -->
<link href="/extra/people.css" rel="

In [1]:
author_lab

NameError: name 'author_lab' is not defined

In [23]:
import pickle
pickle.dump(author_lab, open('author_lab.p', 'wb')) # Store the variable in a file.

In [ ]:
data['146811']['Labs involved']

In [ ]:
data['146811']['Epfl authors']

In [ ]:
# Define an epty set of edges
edges = set()

# For each author
for key in list(dict_authors_set_epfl.keys()):       
    # If the list of co-authors is not empty
    if len(dict_authors_set_epfl[key]) != 0:
        # Create sorted tuples between the author and his coauthors
        new_edges = [tuple(sorted((dictionary_a_id_epfl[key], dictionary_a_id_epfl[co]))) for co in dict_authors_set_epfl[key]]

        # Update the set in order to not have the double arches
        edges.update(new_edges)

In [ ]:
dictionary['catasta michele']

In [ ]:
dictionary_occ['catasta michele']

In [ ]:
G_epfl = nx.MultiGraph()

In [ ]:
G_epfl.add_nodes_from(list(dictionary_id_a_epfl.keys()))

In [ ]:
G_epfl.add_edges_from(list(edges_epfl))

In [ ]:
new_edges = []
to_remove = []
for i,j in list(edges_epfl):
    
    try:    
        for k in list(dictionary_occ[dictionary_id_a_epfl[i]][dictionary_id_a_epfl[j]].keys()):
        #print (list(dictionary_occ[dictionary_id_a_epfl[i]][dictionary_id_a_epfl[j]].keys()))
        #print (i,j)
            new_edges += [(i, j, dict(year = k))]
            to_remove += [(i,j)]
    except:
        continue

In [ ]:
G_epfl.add_edges_from(new_edges)
#G_epfl.remove_edges_from(set(to_remove))

In [ ]:
dictionary_a_id_epfl['agir berker']

In [ ]:
for i,j in G_epfl.edges():
    for k in G_epfl[i][j]:
        #print (k)
        if k == 0:
            G_epfl[i][j][k]['weight'] = dict_numb_coll_epfl[dictionary_id_a_epfl[i]][dictionary_id_a_epfl[j]]
        else:
            #print (G_epfl[i][j][k]['year'])
            G_epfl[i][j][k]['weight'] = dictionary_occ[dictionary_id_a_epfl[i]][dictionary_id_a_epfl[j]][G_epfl[i][j][k]['year']]
    

In [ ]:
G_epfl[dictionary_a_id_epfl['catasta michele']]#[1]#['weight'] = 5

In [ ]:
G[dictionary_a_id_epfl['catasta michele']]

In [ ]:
for i,j in G.edges():
    for k in G[i]:
        #print (k)
        
        G[i][j]['weight'] = dict_numb_coll_epfl[dictionary_id_a_epfl[i]][dictionary_id_a_epfl[j]]

# Basic statistics on the graph

In [ ]:
# Get the range of years
dict_years = defaultdict(int)
for paper in list(data.keys()):
    try:
        #print (paper['Publication date'])
        dict_years[data[paper]['Publication date']] += 1
    except:
        continue

In [ ]:
#dict_years

In [ ]:
del dict_years['August 31-September 2 2005']
del dict_years['0']

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

After a first glance twe observe that:
* The oldest publications are thos which may contain fragmented and incomplete information
* The number of publication in considerably higher from the latest 90s to nowdays

As consequence we decide to focus our attention on the publications corresponding to the aforementioned period.

In [ ]:
plt.bar(dict_years.keys(), dict_years.values(), 0.5, color='g')

In [ ]:
data_cut = {}
for paper in list(data.keys()):
    try:
        if int(data[paper]['Publication date']) >= 1993:
            data_cut[paper] = data[paper]
    except:
        continue

In [ ]:
data_cut['52256']['Labs involved'][0]

# LABS

In [ ]:
# (lab,website)
dict_lab_site = {}
# (lab, num publications)
dict_labs = defaultdict(int)
# (lab, list of papers)
dict_labs_paper = defaultdict(list)
i = 0
for paper in list(data_cut.keys()):
    try:
        if len(data_cut[paper]['Labs involved']) == 1:
            dict_lab_site[list(data_cut[paper]['Labs involved'][0].keys())[0]] = list(data_cut[paper]['Labs involved'][0].values())[0]
            dict_labs[list(data_cut[paper]['Labs involved'][0].keys())[0]] += 1
            dict_labs_paper[list(data_cut[paper]['Labs involved'][0].keys())[0]] += [paper]
            
        else:
            #print ('else')
            for lab in data_cut[paper]['Labs involved']:
                dict_lab_site[list(lab.keys())[0]] = list(lab.values())[0]     
                dict_labs[list(lab.keys())[0]] += 1
                dict_labs_paper[list(lab.keys())[0]] += [paper]
    except:
        continue

# EPFL STRUCTURE

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
site

In [ ]:
req = requests.get('https://search.epfl.ch/ubrowse.action?acro=EPFL')
html = req.content

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
len(soup.findAll('div', {'class':'unit_name'}))

In [ ]:
new_links_to_go = []
dictio = []
for i in soup.findAll('div', {'class':'unit_name'}):
    if i.findAll('a')[0].text.strip()[:6] == 'School':
        new_links_to_go += [i.findAll('a')[0].get('href')]
        dictio += [i.findAll('a')[0].text.strip()]

In [ ]:
new_links_to_go

In [ ]:
dictio

In [ ]:
prova = defaultdict(list)

In [ ]:
def traverse_epfl_tree(root):
    global prova
    
    #Request root url
    request = requests.get(root)
    html = request.content
    #get the soup
    soup = BeautifulSoup(html, 'html.parser')
    
    #find eventual children
    children = soup.findAll("div", { "class" : "unit_name" })
    #check if node is a leaf
    
    is_leaf = (len(children) == 0)
    #print (is_leaf)
    #parse and add node to the db (see def of create_node)
    
    #create_node(soup, is_leaf) # save data 
    
    if is_leaf == False:
        #continue exploring the tree
        for elem in children:
            print (soup.findAll('h2')[0].text.replace('\n',' ').split('  ')[0])
            prova[soup.findAll('h2')[0].text.replace('\n',' ').split('  ')[0]] += [elem.findAll('a')[0].text.strip()]
            #print (elem)
            traverse_epfl_tree("https://search.epfl.ch" + elem.find('a').get('href'))

In [ ]:
#traverse_epfl_tree('https://search.epfl.ch/ubrowse.action?acro=SV')

In [ ]:
import pickle
pickle.dump(prova, open('SV.p', 'wb')) # Store the variable in a file.

In [ ]:
school_SV = pickle.load(open('SV.p', 'rb')) # Recall 

In [ ]:
# Get dictionary (lab,school)
def school_lab(dict_lab_site, name_school):
    
    school = pickle.load(open(name_school + '.p', 'rb')) # Recall 
    lab_school = {}
    for i in list(dict_lab_site.keys()):
        for nodes in list(school.keys()):
            list_node = school[nodes]
            if i in list_node:
                lab_school[i] = name_school
                break
            else:
                continue   
    
    return lab_school

In [ ]:
school_lab(dict_lab_site, 'IC')

# Scrape authors LAB

In [24]:
# (lab,website)
dict_author_site = {}
i = 0
for paper in list(data.keys()):
    try:
        if len(data[paper]['Epfl authors']) == 1:
            dict_author_site[list(data[paper]['Epfl authors'][0].keys())[0]] = list(data[paper]['Epfl authors'][0].values())[0]

        else:
            #print ('else')
            for lab in data[paper]['Epfl authors']:
                dict_author_site[list(lab.keys())[0]] = list(lab.values())[0]  
    except:
        continue